In [158]:
import smtplib
from email.mime.text import MIMEText
from email.header import Header
import requests
from bs4 import BeautifulSoup
from time import sleep
import schedule
import time

In [159]:
def send_email(content):
   # 发件人邮箱和密码,密码在XX邮箱设置中获取
    sender = 'XXXXXXXXXXX'
    password = 'XXXXXXXXX'

    # 收件人邮箱，每天想用邮件接收的邮箱，可以使用多个
    receivers = ['XXXXXXXXX']

    # 邮件内容
    message = MIMEText(content, 'plain', 'utf-8')
    message['From'] = Header('发件人邮箱')
    message['To'] = Header('收件人邮箱', 'utf-8')
    message['Subject'] = Header('GitHub Trending Project', 'utf-8')

    try:
        # QQ邮箱SMTP服务器(smtp.qq.com)，端口为465或587
        smtpObj = smtplib.SMTP_SSL('smtp.qq.com', 465)
        smtpObj.login(sender, password)
        smtpObj.sendmail(sender, receivers, message.as_string())
        print("邮件发送成功")
    except smtplib.SMTPException as e:
        print("邮件发送失败:", str(e))

In [160]:
def get_trending_projects():
    # 发送HTTP请求获取页面内容
    url = 'https://github.com/trending'
    while True:
        try:
            response = requests.get(url)
            break
        except:
            sleep(5)
            print('网络连接失败，正在重试...')
            continue
    html = response.text

    # 使用BeautifulSoup解析页面内容
    soup = BeautifulSoup(html, 'html.parser')

    # 查找项目列表
    repo_list = soup.find_all('article', class_='Box-row')

    # 使用列表保存项目信息
    trending_projects = []

    # 遍历项目列表，获取项目名称、地址和star数量
    for repo in repo_list:


        # 获取项目名称
        name = repo.find('h2').text.split()[-1]

        # 获取项目地址
        url = 'https://github.com' + repo.find(attrs = {'aria-label':'You must be signed in to star a repository'})['href']

        # 获取star数量
        stars = repo.find_all('span')[-1].text.strip()[:-6].split()[0]
        stars = int(stars.replace(',', ''))

        if stars < 300:
            continue

        # 获取项目描述
        description = repo.find('p')
        if description is None:
            description = '无简介'
        else:
            description = description.text.strip()
        
        # 将项目信息添加到列表中
        trending_projects.append('项目名称: ' + name + '\n' + '项目地址: ' + url + '\n' + '项目描述: ' + description + '\n' 
                                     + 'star数量: ' + str(stars) + '\n' + '---' + '\n')

        # # 打印项目信息
        # print('项目名称:', name)
        # print('项目地址:', url)
        # print('项目描述:', description)
        # print('star数量:', stars)
        # print('---')

    # 将trending_projects列表转换成字符串
    trending_projects = ''.join(trending_projects)
    return trending_projects